Primeiramente importamos o panda e lemos a base de dados, bem como fazendo uma primeira checagem do tipo de dados

In [1]:
import pandas as pd

In [2]:
logins = pd.read_parquet('logins.parquet7')

In [3]:
logins.head()

,id,account_id,device_id,installation_id,timestamp,is_from_official_store,is_emulator,has_fake_location_app,has_fake_location_enabled,probable_root,device_age_ms,max_installations_on_related_devices,never_permitted_location_on_account,boot_count,wallpaper_count,n_accounts,ato
0,799740346458,YgXAhVgdDLsozwto,fGb574abkmDJTQ6M,Uv9PEQZEwsW8nLOj,1.621290e+12,1.0,0.0,0.0,0.0,0.0,1.407468e+09,1.0,0.0,129.0,28.0,1.0,0.0
1,799740346459,bO+gPxgfZG9sXtCb,Xll03k9l/OL2r6LM,pR7skP90RuM3oqhK,1.620309e+12,1.0,0.0,0.0,0.0,0.0,1.622747e+10,2.0,0.0,131.0,224.0,1.0,0.0
2,799740346460,9mEBGcdz+RLzRNB4,qoKjqPhxqyALMsQ5,hzKrODiR6F0jG4rH,1.625228e+12,1.0,0.0,0.0,0.0,0.0,2.865500e+10,2.0,0.0,179.0,6.0,1.0,0.0
3,799740346461,tghz/Mlz069esl03,5fqsAP5QUp1M3PLC,krWwUuXbmBEP9g38,1.624645e+12,1.0,0.0,0.0,0.0,0.0,2.484419e+10,3.0,0.0,38.0,11.0,1.0,0.0
4,799740346462,9mEBGcdz+RLzRNB4,qoKjqPhxqyALMsQ5,hzKrODiR6F0jG4rH,1.619907e+12,1.0,0.0,0.0,0.0,0.0,2.334158e+10,2.0,0.0,NaN,NaN,1.0,0.0


In [4]:
print(logins.dtypes)

id                                       object
account_id                               object
device_id                                object
installation_id                          object
timestamp                               float64
is_from_official_store                  float64
is_emulator                             float64
has_fake_location_app                   float64
has_fake_location_enabled               float64
probable_root                           float64
device_age_ms                           float64
max_installations_on_related_devices    float64
never_permitted_location_on_account     float64
boot_count                              float64
wallpaper_count                         float64
n_accounts                              float64
ato                                     float64
dtype: object


In [5]:
print(logins.describe(),"\n")

          timestamp  is_from_official_store   is_emulator  \
count  5.418714e+06            5.418712e+06  5.418710e+06   
mean   1.623015e+12            9.987253e-01  7.197285e-06   
std    2.723952e+09            3.567958e-02  2.682766e-03   
min    1.617814e+12            0.000000e+00  0.000000e+00   
25%    1.620660e+12            1.000000e+00  0.000000e+00   
50%    1.623020e+12            1.000000e+00  0.000000e+00   
75%    1.625521e+12            1.000000e+00  0.000000e+00   
max    1.627614e+12            1.000000e+00  1.000000e+00   

       has_fake_location_app  has_fake_location_enabled  probable_root  \
count           5.418715e+06               5.418710e+06   5.418712e+06   
mean            2.790975e-02               3.557170e+08   1.966349e+08   
std             1.647143e-01               8.017728e+10   5.605361e+10   
min             0.000000e+00               0.000000e+00   0.000000e+00   
25%             0.000000e+00               0.000000e+00   0.000000e+00   
50%   

A partir do describe podemos inferir algumas coisas para o nosso pré-processamento:
- Temos 5.418.714 linhas representando acessos
- Os tipos estão genéricos e podem ser especificados para tipos melhores
    - Id, device_id, account_id e instalation_id podem ser do tipo categoria
    - is_from_official_store, is_emulator, has_fake_location_app, has_fake_location_enabled, probable_root, never_permitted_location_on_account, ato serão do tipo booleano
    - Temos duas colunas com tipo data que podem ser convertidas para data
    - Os demais são tipo númerico
- has_fake_location_enable, probable_root e ato são booleanos porém algumas das linhas tem valores diferentes de 0 e 1 como visto no max do describe, isso quer dizer que esses dados foram erroneamente colocados ali e podem ser considerados ruido, que podem ser removidos
- A coluna wallpaper_count também tem alguns campos com valores negativos, como visto no min do describe, essas linhas serão removidos pois esse tipo de resultado não faz sentido, sendo considerado ruído.
- Algumas das linhas tem informações faltando, NaN e campos vazios, essas linhas serão removidas caso esse campo seja categorico, será atribuido a moda caso seja booleano e a media caso seja numérico.

Observação quanto a esse último ponto, como visto no count, apenas 3390946 dos 5418712 linhas da tabela tem boot_count, remover todas essas linhas pode causar uma grande perda de informação, então a melhor opção no momento seria manter a média mesmo com as mudanças de estatística.
No caso de serem inseridos novos dados é interessante ter a base antes dessa inserção de media nos valores pois pode ser refeito o cálculo antes de inserimos a media dos elementos pode ser refeito pra gerar uma nova média.

In [6]:
logins[logins.ato > 0.0].head(20)

,id,account_id,device_id,installation_id,timestamp,is_from_official_store,is_emulator,has_fake_location_app,has_fake_location_enabled,probable_root,device_age_ms,max_installations_on_related_devices,never_permitted_location_on_account,boot_count,wallpaper_count,n_accounts,ato
5155,799740351613,z71YDYDfVxXhUZty,pK2ugmpFmeD74I7G,U502YG+cNoaHUYFE,1.620843e+12,1.0,0.0,0.0,0.0,0.0,1.037243e+09,1.0,1.0,2.0,2.0,1.0,1.0
5159,799740351617,z71YDYDfVxXhUZty,pK2ugmpFmeD74I7G,U502YG+cNoaHUYFE,1.622917e+12,1.0,0.0,0.0,0.0,0.0,3.098230e+09,1.0,1.0,2.0,2.0,1.0,1.0
5425,799740351883,asz6lASXRKktWDdI,DgT9MGs4PTsbx94l,Pnq09lRTWfELtJ6Z,1.620416e+12,1.0,0.0,0.0,0.0,0.0,0.000000e+00,1.0,0.0,4.0,2.0,NaN,1.0
5857,799740352315,G6q5cv3pS+d5NrRO,yvohplzOvqIfo79L,muYYuaevWlDR78ew,1.623790e+12,1.0,0.0,0.0,0.0,0.0,0.000000e+00,1.0,1.0,6.0,1.0,NaN,1.0
9914,799740356372,erjrTAFd6AsaeQ7c,XgwvQ5vUEjVaHrGe,AJkxDVFnMdqG1vrM,1.620158e+12,1.0,0.0,0.0,0.0,0.0,0.000000e+00,1.0,0.0,NaN,NaN,1.0,1.0
10834,799740357292,dVF08l6z4GcSmufn,LgQ75VFahdAA50v5,oCQFWiefA1yizxan,1.623522e+12,1.0,0.0,0.0,0.0,0.0,0.000000e+00,1.0,1.0,103.0,10.0,NaN,1.0
12646,799740359104,zYmL+56bIde68HMq,rLtNRpOvJ7L7imbx,/W5xM1Q3gNh73QN7,1.623802e+12,1.0,0.0,0.0,0.0,0.0,0.000000e+00,1.0,0.0,38.0,6.0,NaN,1.0
13051,799740359509,YhF++2fQARDM5RRE,0OEam/TgvivGzOqa,/coJ4Lc3QLBTMJjV,1.621453e+12,1.0,0.0,0.0,0.0,0.0,0.000000e+00,1.0,0.0,55.0,12.0,NaN,1.0
13276,799740359734,+W0hBDSHLAu4u79B,PVJFuJLcFjUjgBOA,rMsjA2V4H14EuQSe,1.623603e+12,1.0,0.0,0.0,0.0,0.0,2.319358e+10,4.0,0.0,130.0,27.0,1.0,1.0
15260,799740361718,fW2LWsnXdC9k8Pzq,V3x5praHZqkeZBjD,jhX97rkiQvaG7aUo,1.620311e+12,1.0,0.0,1.0,0.0,0.0,0.000000e+00,1.0,0.0,NaN,NaN,1.0,1.0


In [7]:
clean_logins = logins.copy()
print(clean_logins.dtypes)

id                                       object
account_id                               object
device_id                                object
installation_id                          object
timestamp                               float64
is_from_official_store                  float64
is_emulator                             float64
has_fake_location_app                   float64
has_fake_location_enabled               float64
probable_root                           float64
device_age_ms                           float64
max_installations_on_related_devices    float64
never_permitted_location_on_account     float64
boot_count                              float64
wallpaper_count                         float64
n_accounts                              float64
ato                                     float64
dtype: object


Primeiro vamos remover alguns valores que não fazem sentido, como visto no describe, wallpaper count tem valores negativos, o que não faz sentido, então iremos remover essas linhas

In [8]:
clean_logins.count()

id                                      5418712
account_id                              5418709
device_id                               5418711
installation_id                         5418710
timestamp                               5418714
is_from_official_store                  5418712
is_emulator                             5418710
has_fake_location_app                   5418715
has_fake_location_enabled               5418710
probable_root                           5418712
device_age_ms                           5418714
max_installations_on_related_devices    5418714
never_permitted_location_on_account     5418713
boot_count                              3390946
wallpaper_count                         3390918
n_accounts                              5162027
ato                                     5418714
dtype: int64

In [9]:
negativeValues = clean_logins[clean_logins['wallpaper_count'] < 0 ].index
clean_logins.drop(negativeValues, inplace=True)
clean_logins.count()

id                                      5417465
account_id                              5417462
device_id                               5417464
installation_id                         5417463
timestamp                               5417467
is_from_official_store                  5417465
is_emulator                             5417463
has_fake_location_app                   5417468
has_fake_location_enabled               5417463
probable_root                           5417465
device_age_ms                           5417467
max_installations_on_related_devices    5417467
never_permitted_location_on_account     5417466
boot_count                              3389699
wallpaper_count                         3389671
n_accounts                              5160854
ato                                     5417467
dtype: int64

Inicialmente vamos eliminar as seguintes linhas caso as seguintes colunas não tenha valores (NaN):
id, account_id, device_id, installation_id, timestamp, device_age_ms, ato

Após isso removeremos os valores estranhos das colunas booleanas

Então os seguintes valores NaN serão trocados pela moda:
is_from_official_store, is_emulator, has_fake_location_app, has_fake_location_enabled, probably_root, never_permitted_location_on_accouunt

E os seguintes valores NaN serão trocados pela mediana:
max_installations_on_related_devices, boot_count, wallpaper_count, n_accounts

In [10]:
clean_logins.dropna(subset=["id", "account_id", "device_id", "installation_id", "timestamp", "device_age_ms", "ato"], inplace = True)
clean_logins.count()

id                                      5417441
account_id                              5417441
device_id                               5417441
installation_id                         5417441
timestamp                               5417441
is_from_official_store                  5417437
is_emulator                             5417435
has_fake_location_app                   5417440
has_fake_location_enabled               5417435
probable_root                           5417437
device_age_ms                           5417441
max_installations_on_related_devices    5417439
never_permitted_location_on_account     5417438
boot_count                              3389679
wallpaper_count                         3389651
n_accounts                              5160828
ato                                     5417441
dtype: int64

In [11]:
#Dropping rows with strange value on has_fake_location_enabled
different_values = clean_logins[clean_logins['has_fake_location_enabled'] > 1].index
clean_logins.drop(different_values, inplace=True)

In [12]:
#Dropping rows with strange value on has_fake_location_enabled
different_values2 = clean_logins[clean_logins['probable_root'] > 1].index
clean_logins.drop(different_values2, inplace=True)

In [13]:
#Dropping rows with strange value on has_fake_location_enabled
different_values3 = clean_logins[clean_logins['ato'] > 1].index
clean_logins.drop(different_values3, inplace=True)

In [14]:
clean_logins.count()

id                                      5416990
account_id                              5416990
device_id                               5416990
installation_id                         5416990
timestamp                               5416990
is_from_official_store                  5416986
is_emulator                             5416984
has_fake_location_app                   5416989
has_fake_location_enabled               5416984
probable_root                           5416986
device_age_ms                           5416990
max_installations_on_related_devices    5416988
never_permitted_location_on_account     5416987
boot_count                              3389388
wallpaper_count                         3389360
n_accounts                              5160399
ato                                     5416990
dtype: int64

In [15]:
clean_logins.is_from_official_store.fillna(value=clean_logins.is_from_official_store.mode()[0], inplace=True)

In [16]:
clean_logins.is_emulator.fillna((clean_logins.is_emulator.mode()[0]), inplace=True)

In [17]:
clean_logins.boot_count.fillna((clean_logins.boot_count.mean()), inplace=True)
clean_logins.count()

id                                      5416990
account_id                              5416990
device_id                               5416990
installation_id                         5416990
timestamp                               5416990
is_from_official_store                  5416990
is_emulator                             5416990
has_fake_location_app                   5416989
has_fake_location_enabled               5416984
probable_root                           5416986
device_age_ms                           5416990
max_installations_on_related_devices    5416988
never_permitted_location_on_account     5416987
boot_count                              5416990
wallpaper_count                         3389360
n_accounts                              5160399
ato                                     5416990
dtype: int64

In [18]:
clean_logins.is_from_official_store.fillna(value=clean_logins.is_from_official_store.mode()[0], inplace=True)
clean_logins.is_emulator.fillna((clean_logins.is_emulator.mode()[0]), inplace=True)
clean_logins.has_fake_location_app.fillna((clean_logins.has_fake_location_app.mode()[0]), inplace=True)
clean_logins.has_fake_location_enabled.fillna((clean_logins.has_fake_location_enabled.mode()[0]), inplace=True)
clean_logins.probable_root.fillna((clean_logins.probable_root.mode()[0]), inplace=True)
clean_logins.max_installations_on_related_devices.fillna((clean_logins.max_installations_on_related_devices.mean()), inplace=True)
clean_logins.never_permitted_location_on_account.fillna((clean_logins.never_permitted_location_on_account.mode()[0]), inplace=True)
clean_logins.boot_count.fillna((clean_logins.boot_count.mean()), inplace=True)
clean_logins.wallpaper_count.fillna((clean_logins.wallpaper_count.mean()), inplace=True)
clean_logins.n_accounts.fillna((clean_logins.n_accounts.mean()), inplace=True)
clean_logins.count()

id                                      5416990
account_id                              5416990
device_id                               5416990
installation_id                         5416990
timestamp                               5416990
is_from_official_store                  5416990
is_emulator                             5416990
has_fake_location_app                   5416990
has_fake_location_enabled               5416990
probable_root                           5416990
device_age_ms                           5416990
max_installations_on_related_devices    5416990
never_permitted_location_on_account     5416990
boot_count                              5416990
wallpaper_count                         5416990
n_accounts                              5416990
ato                                     5416990
dtype: int64

Agora com os dados limpos e com a mesma quantiadade, trocamos o tipo de alguns desses dados

In [30]:
clean_logins.id = clean_logins.id.astype('category')
clean_logins.account_id = clean_logins.account_id.astype('category')
clean_logins.device_id = clean_logins.device_id.astype('category')
clean_logins.installation_id = clean_logins.installation_id.astype('category')
clean_logins.timestamp = pd.to_datetime(clean_logins.timestamp, unit='us')
clean_logins.timestamp = clean_logins.timestamp.astype('datetime64[s]')
clean_logins.is_from_official_store = clean_logins.is_from_official_store.astype('bool')
clean_logins.is_emulator = clean_logins.is_emulator.astype('bool')
clean_logins.has_fake_location_app = clean_logins.has_fake_location_enabled.astype('bool')
clean_logins.probable_root = clean_logins.probable_root.astype('bool')
#clean_logins.device_age_ms = pd.to_datetime(clean_logins.device_age_ms, unit='ms')
clean_logins.never_permitted_location_on_account = clean_logins.never_permitted_location_on_account.astype('bool')
clean_logins.ato = clean_logins.ato.astype('bool')
print(clean_logins.dtypes)

id                                            category
account_id                                    category
device_id                                     category
installation_id                               category
timestamp                               datetime64[ns]
is_from_official_store                            bool
is_emulator                                       bool
has_fake_location_app                             bool
has_fake_location_enabled                      float64
probable_root                                     bool
device_age_ms                                  float64
max_installations_on_related_devices           float64
never_permitted_location_on_account               bool
boot_count                                     float64
wallpaper_count                                float64
n_accounts                                     float64
ato                                               bool
dtype: object


In [31]:
clean_logins.head(10)

,id,account_id,device_id,installation_id,timestamp,is_from_official_store,is_emulator,has_fake_location_app,has_fake_location_enabled,probable_root,device_age_ms,max_installations_on_related_devices,never_permitted_location_on_account,boot_count,wallpaper_count,n_accounts,ato
0,799740346458,YgXAhVgdDLsozwto,fGb574abkmDJTQ6M,Uv9PEQZEwsW8nLOj,2021-05-17 22:25:37,True,False,False,0.0,False,1.407468e+09,1.0,False,129.000000,28.000000,1.0,False
1,799740346459,bO+gPxgfZG9sXtCb,Xll03k9l/OL2r6LM,pR7skP90RuM3oqhK,2021-05-06 13:48:57,True,False,False,0.0,False,1.622747e+10,2.0,False,131.000000,224.000000,1.0,False
2,799740346460,9mEBGcdz+RLzRNB4,qoKjqPhxqyALMsQ5,hzKrODiR6F0jG4rH,2021-07-02 12:11:39,True,False,False,0.0,False,2.865500e+10,2.0,False,179.000000,6.000000,1.0,False
3,799740346461,tghz/Mlz069esl03,5fqsAP5QUp1M3PLC,krWwUuXbmBEP9g38,2021-06-25 18:24:27,True,False,False,0.0,False,2.484419e+10,3.0,False,38.000000,11.000000,1.0,False
4,799740346462,9mEBGcdz+RLzRNB4,qoKjqPhxqyALMsQ5,hzKrODiR6F0jG4rH,2021-05-01 22:02:04,True,False,False,0.0,False,2.334158e+10,2.0,False,137.525651,214.928724,1.0,False
5,799740346463,JATYrjV8SNmJaSvR,6vsMTtIB/YcbosQ0,BlzvH1Mhan3uYaGG,2021-06-15 21:38:15,True,False,False,0.0,False,1.946813e+10,11.0,False,137.525651,214.928724,1.0,False
6,799740346464,kfvjQQcAP+QyVeWh,izCoz9U5qm0Zh3mr,1exoSxXx+iHQYm3S,2021-05-10 13:51:17,True,False,False,0.0,False,3.106749e+10,2.0,False,49.000000,51.000000,1.0,False
7,799740346465,bO+gPxgfZG9sXtCb,Xll03k9l/OL2r6LM,pR7skP90RuM3oqhK,2021-05-28 12:37:33,True,False,False,0.0,False,1.813486e+10,2.0,False,131.000000,224.000000,1.0,False
8,799740346466,2NxmkxFB/JqryUO9,amQOo3NgryQSHP91,KhadrOUl+TfvMgAZ,2021-04-17 14:26:15,True,False,False,0.0,False,3.116325e+10,2.0,False,137.525651,214.928724,2.0,False
9,799740346467,0JmTfcV7mLAYCXFZ,+ii227O9GXsUH5FL,10u7zc+xCGVCF8Oh,2021-07-07 18:14:28,True,False,False,0.0,False,2.393548e+10,2.0,False,13.000000,7.000000,1.0,False


In [32]:
clean_logins.to_parquet("clean_logins.parquet")

In [33]:
#%run auxiliar_functions.ipynb
#convert_parquet_to_csv("clean_logins.parquet", "clean_logins.csv")